In [10]:
@activate_coroutine
def coroutine():
    print("begin")
    a = yield 1
    print(a)
    b = yield 2
    print(b)
    print('bye')

In [11]:
a = coroutine()
from inspect import getgeneratorstate

begin


In [8]:
# a.send(3)
getgeneratorstate(a)

'GEN_CLOSED'

In [9]:
from functools import wraps


def activate_coroutine(func):
    @wraps(func)
    def wrapper(*arg, **kwargs):
        gen = func(*arg, **kwargs)
        next(gen)
        return gen

    return wrapper

In [23]:
class DemoException(Exception):
    pass


def demo_exc_handler():
    print('-> coroutine started')
    while True:
        try:
            x = yield
        except GeneratorExit:
            print('*** DemoException handled. Continuing...')
            return 2
        else:
            print('-> coroutine received: {!r}'.format(x))

In [24]:
exe_coro = demo_exc_handler()
next(exe_coro)
a = exe_coro.close()
a

-> coroutine started
*** DemoException handled. Continuing...


In [25]:
def gen():
    yield from 'abc'
    yield from 'def'


list(gen())

['a', 'b', 'c', 'd', 'e', 'f']

In [4]:
from collections import namedtuple

Result = namedtuple('Result', 'cnt avg')


def avg():
    total = 0
    cnt = 0
    avg = None
    while True:
        term = yield
        if term is None:
            break
        total += term
        cnt += 1
        avg = total / cnt
    return Result(cnt, avg)


def grouper(result, key):
    while True:
        result[key] = yield from avg()


def main(data):
    results = {}
    for key, values in data.items():
        group = grouper(results, key)
        next(group)
        for value in values:
            group.send(value)

        group.send(None)

    print(results)
    report(results)


data = {
    'girls;kg':
        [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
    'girls;m':
        [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
    'boys;kg':
        [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
    'boys;m':
        [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46],
}


def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print('{:2} {:5} averaging {:.2f}{}'.format(
            result.cnt, group, result.avg, unit))


main(data)

{'girls;kg': Result(cnt=10, avg=42.040000000000006), 'girls;m': Result(cnt=10, avg=1.4279999999999997), 'boys;kg': Result(cnt=9, avg=40.422222222222224), 'boys;m': Result(cnt=9, avg=1.3888888888888888)}
 9 boys  averaging 40.42kg
 9 boys  averaging 1.39m
10 girls averaging 42.04kg
10 girls averaging 1.43m


In [12]:
def gen123():
    a = yield 1
    print(a)
    return 2
    yield 3


a = gen123()

In [6]:
next(a)

1

In [8]:
try:
    next(a)
except Exception as e:
    print(e.value)

None


In [15]:
from inspect import getgeneratorstate

getgeneratorstate(a)

a = gen123()
next(a)
next(a)

None


StopIteration: 2